In [2]:
import numpy as np
import tensorflow
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint

In [3]:
train_1=np.load('train_5class_1800.npy', allow_pickle=True, encoding='bytes')

In [4]:
train_ext=np.load('train_extra_added_200.npy', allow_pickle=True, encoding='bytes')

In [5]:
train=np.concatenate((train_1, train_ext), axis=0)

In [6]:
val=np.load('val_5class_200.npy', allow_pickle=True, encoding='bytes')

In [7]:
test=np.load('test_5class_500.npy', allow_pickle=True, encoding='bytes')

In [8]:
np.random.shuffle(train)

In [9]:
np.random.shuffle(test)

In [10]:
np.random.shuffle(val)

In [11]:
# preprocessing train data

n=len(train)
x_train = train[:, :-1]
y_train = train[:, -1]

g = []
for i in range(n):
    a=x_train[i][0]
    g.append(a)
    
train_x = np.array(g)

In [12]:
#preprocessing test data

m=len(test)
x_test = test[:, :-1]
y_test = test[:, -1]

h=[]
for i in range(m):
    b=x_test[i][0]
    h.append(b)
    
test_x = np.array(h)


In [13]:
# preprocessing validation data

o=len(val)
x_val = val[:, :-1]
y_val = val[:, -1]

k=[]
for i in range(o):
    c=x_val[i][0]
    k.append(c)
    
val_x = np.array(k)

In [14]:
import category_encoders as ce
enc = ce.OneHotEncoder(y_train, handle_unknown='return_nan', return_df=True, use_cat_names=True)

In [15]:
y_train = enc.fit_transform(y_train)

In [16]:
y_test = enc.transform(y_test)

In [17]:
y_val = enc.transform(y_val)

In [18]:
train_x.shape

(2000, 432, 504, 3)

In [22]:
cnn = Sequential()

cnn.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(432, 504, 3))) #1

cnn.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #2
cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #3
cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #4
cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #5
# cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(678, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #6
# cnn.add(MaxPool2D(pool_size=(2,2)))

cnn.add(Conv2D(1024, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #7
# cnn.add(MaxPool2D(pool_size=(2,2)))

# cnn.add(Conv2D(1024, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) #8
# cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

# cnn.add(Conv2D(360, kernel_size=(3,3), strides=(3,3), padding='same', activation='relu')) #9
# cnn.add(Conv2D(400, kernel_size=(3,3), strides=(3,3), padding='same', activation='relu')) #10
# cnn.add(MaxPool2D(pool_size=(2,2)))
# cnn.add(Dropout(0.25))

# cnn.add(Conv2D(430, kernel_size=(3,3), strides=(3,3), padding='same', activation='relu')) #11
# cnn.add(Conv2D(480, kernel_size=(3,3), strides=(3,3), padding='same', activation='relu')) #12
# cnn.add(MaxPool2D(pool_size=(2,2)))
# cnn.add(Dropout(0.25))

# cnn.add(Conv2D(510, kernel_size=(3,3), strides=(3,3), padding='same', activation='relu')) #13
# cnn.add(Conv2D(550, kernel_size=(3,3), strides=(3,3), padding='same', activation='relu')) #14
# cnn.add(Conv2D(610, kernel_size=(3,3), strides=(3,3), padding='same', activation='relu')) #15
# cnn.add(Dropout(0.25))


cnn.add(GlobalAveragePooling2D())
# cnn.add(Flatten())

cnn.add(Dense(550, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(330, activation='relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(170, activation='relu'))
cnn.add(Dropout(0.25))
cnn.add(Dense(40, activation='relu'))
cnn.add(Dropout(0.25))

cnn.add(Dense(5, activation='softmax'))

In [23]:
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 432, 504, 32)      896       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 432, 504, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 216, 252, 64)      0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 216, 252, 64)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 216, 252, 128)     73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 108, 126, 128)     0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 108, 126, 128)    

In [24]:
cnn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [25]:
filepath = "saved-model-{epoch:02d}-{val_loss:.2f}.hdf5"

In [26]:
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='max')

In [ ]:
cnn.fit(train_x, y_train, batch_size=8, epochs=100, validation_data=(val_x, y_val), verbose=1, callbacks=[checkpoint])

Epoch 1/100


In [23]:
# cnn.fit(train_x, y_train, batch_size=8, epochs=100, validation_data=(val_x, y_val), verbose=1, callbacks=[checkpoint])

Epoch 1/100
250/250 [==============================] - ETA: 0s - loss: 1.6102 - accuracy: 0.1880
Epoch 00001: saving model to saved-model-01-1.61.hdf5
250/250 [==============================] - 251s 1s/step - loss: 1.6102 - accuracy: 0.1880 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 2/100
250/250 [==============================] - ETA: 0s - loss: 1.6101 - accuracy: 0.1775
Epoch 00002: saving model to saved-model-02-1.61.hdf5
250/250 [==============================] - 234s 936ms/step - loss: 1.6101 - accuracy: 0.1775 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 1.6099 - accuracy: 0.1915
Epoch 00003: saving model to saved-model-03-1.61.hdf5
250/250 [==============================] - 230s 921ms/step - loss: 1.6099 - accuracy: 0.1915 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 4/100
250/250 [==============================] - ETA: 0s - loss: 1.6099 - accuracy: 0.1955
Epoch 00004: saving model to saved-model-04-1.61

Epoch 30/100
250/250 [==============================] - ETA: 0s - loss: 1.6099 - accuracy: 0.1850
Epoch 00030: saving model to saved-model-30-1.61.hdf5
250/250 [==============================] - 231s 922ms/step - loss: 1.6099 - accuracy: 0.1850 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 31/100
250/250 [==============================] - ETA: 0s - loss: 1.6099 - accuracy: 0.1850
Epoch 00031: saving model to saved-model-31-1.61.hdf5
250/250 [==============================] - 230s 920ms/step - loss: 1.6099 - accuracy: 0.1850 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 32/100
250/250 [==============================] - ETA: 0s - loss: 1.6099 - accuracy: 0.1920
Epoch 00032: saving model to saved-model-32-1.61.hdf5
250/250 [==============================] - 230s 922ms/step - loss: 1.6099 - accuracy: 0.1920 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 33/100
250/250 [==============================] - ETA: 0s - loss: 1.6099 - accuracy: 0.1835
Epoch 00033: saving model to saved-model-

KeyboardInterrupt: 